In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
import ipywidgets as widgets
import cv2
from czifile import CziFile

path = "../"

with CziFile(join(path, "test2.czi")) as czi:
    image_arrays = czi.asarray()
    meta = czi.metadata(raw=False)
    
#display(meta)

TODO
- Calibrate h, templateWidowSize and serachWindowSize  (Denoising)
- Calibrate threshold (Thresholding)
- Understand how distances are calculated and what is the watershed marking

In [ ]:
def normalize_image(img):
    img = (img/img.max())*255
    return img.astype('uint8')

def denoise_image(img, h):
    # Filtro mediana para eliminar ruido impulsivo
    img = cv2.medianBlur(img, 5)
    # Non-local mean denoising http://www.ipol.im/pub/art/2011/bcm_nlm/
    img = cv2.fastNlMeansDenoising(img, None, h=h, templateWindowSize=7, searchWindowSize=21)
    return img

def get_binary_mask(img):
    # Adaptive thresholding    
    binary_mask = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                        cv2.THRESH_BINARY, 21, 0)
    # Cerradura morfologica
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, 
                                   kernel=np.ones((3,3),np.uint8), iterations = 4)
    return binary_mask

def get_watershed(dist, threshold=0.3):
    ret, sure_fg = cv2.threshold(dist, threshold*dist.max(), 255, 0)
    ret, markers = cv2.connectedComponents(sure_fg.astype('uint8'))
    watershed = cv2.watershed(cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), markers)
    return watershed

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(6, 5), tight_layout=True, sharex=True, sharey=True)

def update(k, z, h):
    img = normalize_image(image_arrays[0, 0, 0, k, z, 200:600, 200:600, 0])
    ax[0, 0].imshow(img, cmap=plt.cm.Greys_r)
    # Denoising
    img = denoise_image(img, h)
    ax[0, 1].imshow(img, cmap=plt.cm.Greys_r)
    # Binarization
    binary_mask = ~get_binary_mask(img)
    ax[0, 2].imshow(binary_mask, cmap=plt.cm.Greys_r)
    # Distance transform
    dist_transform = cv2.distanceTransform(binary_mask, cv2.DIST_L2, 5)
    ax[1, 0].imshow(dist_transform, cmap=plt.cm.Greys_r)
    # Watershed transform
    watershed = get_watershed(dist_transform, threshold=0.1)
    ax[1, 1].imshow(watershed, cmap=plt.cm.tab20)
    # Original with watershed borders on top
    img2 = img.copy()
    img2[watershed==-1] = 255
    ax[1, 2].imshow(img2, cmap=plt.cm.Greys_r)    
    display(np.unique(watershed))
    

widgets.interact(update, 
                 k=widgets.IntSlider(min=0, max=59),
                 z=widgets.IntSlider(min=0, max=16),
                 h=widgets.FloatSlider(min=0., max=100., value=30, step=0.01)
                 );